### Imports

In [1]:
import torch
import torch.nn as nn
import torch_directml
import torchmetrics
import torch.optim as optim
import torchmetrics.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')
#AMD GPU Acceleration
device = torch_directml.device(torch_directml.default_device())

C:\Users\cafes\Envs\torch-dml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
set_seed()

In [4]:
train_writer = SummaryWriter('.\\runs\\train\\gru\\')
val_writer = SummaryWriter('.\\runs\\validate\\gru\\')

#### Read in Train/test

In [5]:
train_df = pd.read_csv('train.csv', index_col='Time')
test_df = pd.read_csv('test.csv', index_col='Time')
train_df

,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,Test_Visit,Event
Time,,,,,,,,,
0,-0.416921,0.680190,-0.585780,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
1,-0.420816,0.678475,-0.578919,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
2,-0.419886,0.676579,-0.580286,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
3,-0.413896,0.674399,-0.594393,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
4,-0.403594,0.674941,-0.607099,0.866530,-0.110647,-1.719166,-0.182777,0.648921,0
...,...,...,...,...,...,...,...,...,...
136,-0.230050,0.202884,-1.195147,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0
137,-0.219754,0.198519,-1.206719,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0
138,-0.210724,0.204065,-1.211322,-1.154029,-0.487734,0.581678,1.891825,-0.684845,0


In [6]:
test_df

,AccV,AccML,AccAP,Medication,Age,Sex,YearsSinceDx,Test_Visit
Time,,,,,,,,
0,-9.541841,0.572790,-1.409475,0.86653,-0.110647,-1.719166,-0.182777,0.648921
1,-9.544045,0.570599,-1.436634,0.86653,-0.110647,-1.719166,-0.182777,0.648921
2,-9.537242,0.568221,-1.425318,0.86653,-0.110647,-1.719166,-0.182777,0.648921
3,-9.539138,0.570690,-1.411444,0.86653,-0.110647,-1.719166,-0.182777,0.648921
4,-9.548736,0.568310,-1.425458,0.86653,-0.110647,-1.719166,-0.182777,0.648921
...,...,...,...,...,...,...,...,...
281683,-0.896122,-0.228349,0.397413,0.86653,1.272005,-1.719166,-0.342362,-0.792988
281684,-0.898800,-0.228733,0.398382,0.86653,1.272005,-1.719166,-0.342362,-0.792988
281685,-0.898516,-0.227567,0.400168,0.86653,1.272005,-1.719166,-0.342362,-0.792988


In [ ]:
df = pd.read_csv('subset.csv', index_col='Time')
test_df = pd.read_csv('test_subset.csv', index_col='Time')

In [6]:
train = df.drop(columns=['Subject', 'Id'])
test = test_df.drop(columns=['Subject', 'Id'])

In [7]:
train

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx,Event
Time,,,,,,,,,,
0,1.331028,1.883634,1.260774,3,2.0,1,68,0,9.0,0
1,1.400329,1.858220,1.286215,3,2.0,1,68,0,9.0,0
2,1.448553,1.878501,1.299326,3,2.0,1,68,0,9.0,0
3,1.452512,1.845912,1.334272,3,2.0,1,68,0,9.0,0
4,1.355745,1.808378,1.362725,3,2.0,1,68,0,9.0,0
...,...,...,...,...,...,...,...,...,...,...
13940,-0.150027,1.096938,-0.981169,1,0.0,1,75,1,26.0,0
13941,-0.142915,1.099812,-0.983866,1,0.0,1,75,1,26.0,0
13942,-0.123513,1.118531,-0.982284,1,0.0,1,75,1,26.0,0


In [8]:
test

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx
Time,,,,,,,,,
0,-0.236636,0.500452,-0.519794,3,2.0,1,68,0,9.0
1,-0.238462,0.498884,-0.528549,3,2.0,1,68,0,9.0
2,-0.232826,0.497181,-0.524901,3,2.0,1,68,0,9.0
3,-0.234396,0.498949,-0.520429,3,2.0,1,68,0,9.0
4,-0.242348,0.497245,-0.524946,3,2.0,1,68,0,9.0
...,...,...,...,...,...,...,...,...,...
281683,6.926261,-0.073130,0.062637,2,0.0,1,79,0,8.0
281684,6.924043,-0.073405,0.062949,2,0.0,1,79,0,8.0
281685,6.924278,-0.072570,0.063525,2,0.0,1,79,0,8.0


In [5]:
df = pd.read_csv('downsample30.csv')
test_df = pd.read_csv('test_downsample.csv')

In [6]:
train = df.drop(columns=['Subject', 'Id']).set_index('Time')
test = test_df.drop(columns=['Subject', 'Id']).set_index('Time')
train

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx,Event
Time,,,,,,,,,,
0,-0.420712,0.673434,-0.591752,3,2.0,1,68,0,9.0,0
1,-0.424601,0.671724,-0.584907,3,2.0,1,68,0,9.0,0
2,-0.423672,0.669832,-0.586271,3,2.0,1,68,0,9.0,0
3,-0.417690,0.667659,-0.600345,3,2.0,1,68,0,9.0,0
4,-0.407401,0.668199,-0.613021,3,2.0,1,68,0,9.0,0
...,...,...,...,...,...,...,...,...,...,...
136,-0.234080,0.197408,-1.199697,20,1.0,0,65,1,22.0,0
137,-0.223797,0.193055,-1.211242,20,1.0,0,65,1,22.0,0
138,-0.214779,0.198586,-1.215835,20,1.0,0,65,1,22.0,0


In [7]:
test

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx
Time,,,,,,,,,
0,-9.533939,0.566322,-1.413525,3,2.0,1,68,0,9.0
1,-9.536140,0.564137,-1.440621,3,2.0,1,68,0,9.0
2,-9.529345,0.561765,-1.429331,3,2.0,1,68,0,9.0
3,-9.531239,0.564227,-1.415490,3,2.0,1,68,0,9.0
4,-9.540825,0.561854,-1.429471,3,2.0,1,68,0,9.0
...,...,...,...,...,...,...,...,...,...
281683,-0.899299,-0.232668,0.389147,2,0.0,1,79,0,8.0
281684,-0.901973,-0.233051,0.390114,2,0.0,1,79,0,8.0
281685,-0.901690,-0.231888,0.391896,2,0.0,1,79,0,8.0


### Dataset/DataLoader

In [7]:
class ParkinsonDataset(Dataset):
    def __init__(self, df, lookback=15, goal='train'):
        super().__init__()
        self.data = df.to_numpy()
        self.lookback = lookback
        self.goal = goal

    # Return length of sample
    def __len__(self):
        return self.data.shape[0]

    # Get data at idx
    def __getitem__(self, idx):
        if self.goal == 'train':
            if idx < self.lookback:
                #Pad with zeros
                features = torch.zeros(self.lookback, self.data.shape[1]-1, dtype=torch.float32)
                features[idx] = torch.tensor(self.data[idx, :-1], dtype=torch.float32)
                label = torch.tensor(self.data[idx, -1], dtype=torch.float32)
            else:
                #Otherwise, take the last lookback samples
                features = torch.tensor(self.data[idx - self.lookback:idx, :-1], dtype=torch.float32)
                label = torch.tensor(self.data[idx, -1], dtype=torch.float32)

            features = features.to(device)
            label = label.to(device)
            return features, label

        else:
            if idx < self.lookback:
                #If the index is less than lookback, pad with zeros
                features = torch.zeros(self.lookback, self.data.shape[1], dtype=torch.float32)
                features[idx] = torch.tensor(self.data[idx, :], dtype=torch.float32)
            else:
                #Otherwise, take the last lookback samples
                features = torch.tensor(self.data[idx - self.lookback:idx, :], dtype=torch.float32)

            features = features.to(device)
            return features

In [8]:
def train_val_split(df):
    #Split the training dataset into training/validation sets
    #80% Training - 20% Validation
    train_size = int(.8 * len(df))
    val_size = len(df)-train_size
    train_data, val_data = torch.utils.data.random_split(df, [train_size, val_size])

    #Instantiate DataLoaders
    train_loader = DataLoader(train_data,
                              batch_size=1024,
                              shuffle=False)

    val_loader = DataLoader(val_data,
                            batch_size=1024,
                            shuffle=False)
    
    return train_loader, val_loader

In [9]:
train = ParkinsonDataset(train_df, goal='train')
train_loader, val_loader = train_val_split(train)

In [ ]:
test = test_df.drop(columns='Id')
test = ParkinsonDataset(test, goal='test')
test_loader = DataLoader(test, 
                         batch_size=1024, 
                         shuffle=False)

In [10]:
#Creating accuracy metric using torchmetrics
metric = torchmetrics.AveragePrecision(task='multiclass', num_classes=4, average='weighted', thresholds=4).to(device)

### Overfit Batch Function

In [17]:
#Function to determine whether the model is training appropriately
def overfit_batch(model, criterion, optimizer, batch, epochs=100):
    model = model.to(device)
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        for _ in range(len(batch)):
            features, label = batch
            features = features.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            logits = model(features)
            loss = criterion(logits, label.long())
            ypred = torch.softmax(logits, dim=1)
            metric(ypred, label.long())
            precision=metric.compute()
            
            loss.backward()
            optimizer.step()

            loss_total += loss.item()
            precision_total += precision.item()

        avg_loss = loss_total / len(batch)
        avg_precision = precision_total / len(batch)
        if epoch %10 == 9:
            print(f'Epoch [{epoch + 1}/{epochs}] | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}')

### Train | Validate | Predict

In [12]:
def predict(model, dataloader):
    predictions = []
    with tqdm(dataloader, colour='yellow') as pbar:
        for features in pbar:
            features = features.to(device)
            preds = model(features)
            preds = torch.argmax(preds, dim=1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions

In [13]:
#Validate one epoch
def validate(model, criterion, dataloader, epoch):
    model.eval()
    metric.reset()
    
    loss_total = 0.0
    precision_total = 0.0
    
    with tqdm(dataloader, desc='Validate', total=len(dataloader), unit='batch', colour='blue') as pbar:
        with torch.no_grad():
            for batch in pbar:
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                logits = model(features)               
                #Update running loss and precision
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                precision_total += precision.item()
                loss_total += loss.item()

                #Update the tqdm progress bar description with loss and precision
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')
                
        avg_loss = loss_total / len(dataloader)
        avg_precision = precision_total / len(dataloader)
        
    model.train()    
    return avg_loss, avg_precision

In [24]:
def train_validate(model, criterion, optimizer, scheduler, epochs=100, train_loader=train_loader, val_loader=val_loader, patience=5):
    model = model.train()
    best_precision = None
    min_delta = 0
    counter = 0
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        with tqdm(train_loader, desc='Train', total=len(train_loader), unit='batch', colour='green') as pbar:
            for batch in pbar:
                #Set features/targets to device
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                #Predict labels
                optimizer.zero_grad()                
                logits = model(features)
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                #Tensorboard
                train_writer.add_scalar('Train-Loss', loss, epoch)
                train_writer.add_scalar('Train-Precision', precision, epoch)
                
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')

                #Adjust gradient
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                #Accumulate total loss and precision
                loss_total += loss.item()
                precision_total += precision.item()

            avg_loss = loss_total / len(train_loader)
            avg_precision = precision_total / len(train_loader)
         
        #Report training loss/precision
        print(f'Epoch {epoch + 1} | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}%')

        if epoch%3==2:
            #Validate one epoch and update lr scheduler
            avg_val_loss, avg_val_precision = validate(model, criterion, val_loader, epoch)
            scheduler.step(avg_val_precision)
        
            #Early Stopping
            if best_precision == None:
                best_precision = avg_val_precision
            elif(best_precision - avg_val_precision < min_delta):
                best_precision = avg_val_precision
                counter = 0
            elif(best_precision - avg_val_precision > min_delta):
                counter += 1
                if(counter >= patience):
                    print(f'Early Stopping: Precision stagnated or decreased too many times. Best Validation: {(best_precision*100.0):.2f}%')
                    break

            print(f'Epoch {epoch + 1} | Avg Loss: {avg_val_loss:.4f} | Avg Precision: {(avg_val_precision * 100.0):.2f}% | Patience: {counter}')

    train_writer.flush()
    train_writer.close()
    val_writer.flush()
    val_writer.close()
    print(f'Training Complete - Best Validate Precision: {(best_precision*100.0):.2f}%')
    
    return model

### Model Architecture

In [15]:
class Gru(nn.Module):
    def __init__(self, num_inputs, hidden_size, num_layers, num_outputs):
        super().__init__()
        self.hidden = hidden_size
        self.layers = num_layers
        self.gru = nn.GRU(num_inputs,
                            hidden_size,
                            num_layers,
                            batch_first=True)
        
        # Initialize weights
        for name, param in self.gru.named_parameters():
            if 'weight' in name:
                nn.init.kaiming_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0.0)

        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_size, num_outputs)
        nn.init.xavier_uniform_(self.fc1.weight)
        
    def forward(self, x):
        h0 = torch.zeros((self.layers, x.size(0), self.hidden), dtype=torch.float32).to(device)
        out, _ = self.gru(x, h0)
        out = out[:, -1, :]

        # Apply batch normalization
        out = self.batch_norm1(out)
        out = self.elu(out)
        out = self.fc1(out)  
        return out

### Overfit

In [19]:
batch = next(iter(train_loader))
model = Gru(8, 500, 1, 4)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs = 100

overfit_batch(model, criterion, optimizer, batch, epochs)

Epoch [10/100] | Avg Loss: 0.1739 | Avg Precision: 94.62
Epoch [20/100] | Avg Loss: 0.0294 | Avg Precision: 99.75
Epoch [30/100] | Avg Loss: 0.0215 | Avg Precision: 99.56
Epoch [40/100] | Avg Loss: 0.0055 | Avg Precision: 100.00
Epoch [50/100] | Avg Loss: 0.0011 | Avg Precision: 100.00
Epoch [60/100] | Avg Loss: 0.0006 | Avg Precision: 100.00
Epoch [70/100] | Avg Loss: 0.0004 | Avg Precision: 100.00
Epoch [80/100] | Avg Loss: 0.0003 | Avg Precision: 100.00
Epoch [90/100] | Avg Loss: 0.0002 | Avg Precision: 100.00
Epoch [100/100] | Avg Loss: 0.0002 | Avg Precision: 100.00


# Training

### 11-3-23

#### Model 7 - Subset dataset

In [19]:
model = Gru(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.02)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.8, patience=1, min_lr=.001)
epochs = 10

model7 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model7.state_dict(), '.\\models\\subset\\gru\\model7.pth')

Train: 100%|██████████████████████| 3560/3560 [09:55<00:00,  5.97batch/s, Loss: 0.3601 | Precision: 79.63%]


Epoch 1 | Avg Loss: 0.4144 | Avg Precision: 77.88%


Train: 100%|██████████████████████| 3560/3560 [10:06<00:00,  5.87batch/s, Loss: 0.3424 | Precision: 81.63%]


Epoch 2 | Avg Loss: 0.3829 | Avg Precision: 81.47%


Train: 100%|██████████████████████| 3560/3560 [10:08<00:00,  5.85batch/s, Loss: 0.3592 | Precision: 81.99%]


...

Epoch 9 | Avg Loss: 0.3740 | Avg Precision: 82.49%


Validate: 100%|█████████████████████| 890/890 [02:16<00:00,  6.51batch/s, Loss: 0.3181 | Precision: 81.66%]


Epoch 9 | Avg Loss: 0.3842 | Avg Precision: 81.61% | Patience: 1


Train: 100%|██████████████████████| 3560/3560 [09:32<00:00,  6.22batch/s, Loss: 0.3563 | Precision: 82.51%]

Epoch 10 | Avg Loss: 0.3732 | Avg Precision: 82.48%
Training Complete


#### Model 8 - Downsample dataset - Lookback 100

In [35]:
model = Gru(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.02)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=2, min_lr=.001)
epochs = 100

model8 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model8.state_dict(), '.\\models\\downsample\\gru\\model8.pth')

Train: 100%|████████████████████████| 295/295 [01:39<00:00,  2.98batch/s, Loss: 0.2880 | Precision: 74.94%]


Epoch 1 | Avg Loss: 0.5076 | Avg Precision: 72.73%


Train: 100%|████████████████████████| 295/295 [01:29<00:00,  3.28batch/s, Loss: 0.2476 | Precision: 77.45%]


Epoch 2 | Avg Loss: 0.4512 | Avg Precision: 76.75%


Train: 100%|████████████████████████| 295/295 [01:29<00:00,  3.29batch/s, Loss: 0.2614 | Precision: 78.70%]


...

Epoch 44 | Avg Loss: 0.3941 | Avg Precision: 80.77%


Train: 100%|████████████████████████| 295/295 [01:26<00:00,  3.40batch/s, Loss: 0.1775 | Precision: 81.02%]


Epoch 45 | Avg Loss: 0.3940 | Avg Precision: 81.08%


Validate: 100%|███████████████████████| 74/74 [00:15<00:00,  4.81batch/s, Loss: 0.4260 | Precision: 79.73%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 81.84%
Training Complete


### 11-5-23

#### Model 9 - Lookback 30 - LR adjust

In [21]:
model = Gru(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)


model9 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model9.state_dict(), '.\\models\\downsample\\gru\\model9.pth')

Train: 100%|████████████████████████| 295/295 [00:56<00:00,  5.19batch/s, Loss: 0.5077 | Precision: 75.69%]


Epoch 1 | Avg Loss: 0.4896 | Avg Precision: 72.25%


Train: 100%|████████████████████████| 295/295 [00:56<00:00,  5.22batch/s, Loss: 0.4472 | Precision: 80.78%]


Epoch 2 | Avg Loss: 0.3948 | Avg Precision: 80.19%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  6.05batch/s, Loss: 0.4732 | Precision: 81.13%]


...

Epoch 99 | Avg Loss: 0.2803 | Avg Precision: 87.47%


Train: 100%|████████████████████████| 295/295 [00:56<00:00,  5.23batch/s, Loss: 0.2192 | Precision: 87.48%]


Epoch 100 | Avg Loss: 0.2802 | Avg Precision: 87.48%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  6.04batch/s, Loss: 0.2867 | Precision: 87.41%]

Epoch 100 | Avg Loss: 0.2841 | Avg Precision: 87.40% | Patience: 0
Training Complete - Best Precision: 87.40%


#### Model 10 - Added ELU before Linear Layer

In [23]:
model = Gru(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)


model10 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model10.state_dict(), '.\\models\\downsample\\gru\\model10.pth')

Train: 100%|████████████████████████| 295/295 [01:04<00:00,  4.60batch/s, Loss: 0.5572 | Precision: 76.16%]


Epoch 1 | Avg Loss: 0.4847 | Avg Precision: 72.60%


Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.62batch/s, Loss: 0.4972 | Precision: 80.49%]


Epoch 2 | Avg Loss: 0.4043 | Avg Precision: 80.08%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  6.07batch/s, Loss: 0.4601 | Precision: 77.54%]


...

Epoch 99 | Avg Loss: 0.2768 | Avg Precision: 87.42%


Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.67batch/s, Loss: 0.1434 | Precision: 87.41%]


Epoch 100 | Avg Loss: 0.2766 | Avg Precision: 87.43%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  6.01batch/s, Loss: 0.2849 | Precision: 87.10%]

Epoch 100 | Avg Loss: 0.2841 | Avg Precision: 86.99% | Patience: 3
Training Complete - Best Precision: 87.11%


### 11-9-23

#### Model 11 - Lookback 15 - Batch 512

In [ ]:
model = Gru(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)


model11 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model11.state_dict(), '.\\models\\downsample\\gru\\model11.pth')

Train: 100%|████████████████████████| 589/589 [01:11<00:00,  8.28batch/s, Loss: 0.4854 | Precision: 76.62%]


Epoch 1 | Avg Loss: 0.4717 | Avg Precision: 73.65%


Train: 100%|████████████████████████| 589/589 [01:10<00:00,  8.32batch/s, Loss: 0.4844 | Precision: 79.99%]


Epoch 2 | Avg Loss: 0.4096 | Avg Precision: 79.46%


Train: 100%|████████████████████████| 589/589 [01:10<00:00,  8.34batch/s, Loss: 0.3935 | Precision: 81.11%]


...

Train: 100%|████████████████████████| 589/589 [01:10<00:00,  8.32batch/s, Loss: 0.3362 | Precision: 86.04%]


Epoch 95 | Avg Loss: 0.3051 | Avg Precision: 85.90%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.93batch/s, Loss: 0.3161 | Precision: 85.34%]


Epoch 95 | Avg Loss: 0.3076 | Avg Precision: 85.33% | Patience: 7


Train:  90%|█████████████████████▋  | 532/589 [01:04<00:06,  8.23batch/s, Loss: 0.2914 | Precision: 86.08%]

### 11-10-23

#### Model 1 - Lookback 15 - Batch 1024

In [20]:
model = Gru(9, 300, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001, verbose=True)


model1 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model1.state_dict(), '.\\models\\downsample\\gru\\model1.pth')

Train: 100%|████████████████████████| 295/295 [01:11<00:00,  4.11batch/s, Loss: 0.3089 | Precision: 77.75%]


Epoch 1 | Avg Loss: 0.4506 | Avg Precision: 74.46%


Train: 100%|████████████████████████| 295/295 [01:04<00:00,  4.60batch/s, Loss: 0.3417 | Precision: 81.46%]


Epoch 2 | Avg Loss: 0.3841 | Avg Precision: 80.86%


Train: 100%|████████████████████████| 295/295 [01:04<00:00,  4.59batch/s, Loss: 0.3448 | Precision: 82.43%]


...

Epoch 99 | Avg Loss: 0.2812 | Avg Precision: 87.00%


Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.61batch/s, Loss: 0.3050 | Precision: 87.01%]


Epoch 100 | Avg Loss: 0.2823 | Avg Precision: 87.02%


Validate: 100%|███████████████████████| 74/74 [00:13<00:00,  5.65batch/s, Loss: 0.3245 | Precision: 87.18%]

Epoch 100 | Avg Loss: 0.2913 | Avg Precision: 86.98% | Patience: 0
Training Complete - Best Precision: 86.98%


#### Model 2 - Lookback 30 - Batch 1024 - Changed Weight initializations

In [23]:
model = Gru(9, 350, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001, verbose=True)
epochs=200

model2 = train_validate(model, criterion, optimizer, scheduler, epochs)
torch.save(model2.state_dict(), '.\\models\\downsample\\gru\\model2.pth')

Train: 100%|████████████████████████| 295/295 [01:18<00:00,  3.77batch/s, Loss: 0.3509 | Precision: 77.55%]


Epoch 1 | Avg Loss: 0.4538 | Avg Precision: 74.57%


Train: 100%|████████████████████████| 295/295 [01:17<00:00,  3.79batch/s, Loss: 0.4633 | Precision: 81.08%]


Epoch 2 | Avg Loss: 0.3924 | Avg Precision: 80.54%


Train: 100%|████████████████████████| 295/295 [01:18<00:00,  3.78batch/s, Loss: 0.4140 | Precision: 82.14%]


...

Epoch 199 | Avg Loss: 0.2647 | Avg Precision: 88.01%


Train: 100%|████████████████████████| 295/295 [01:18<00:00,  3.75batch/s, Loss: 0.2467 | Precision: 88.07%]


Epoch 200 | Avg Loss: 0.2640 | Avg Precision: 88.08%


Validate: 100%|███████████████████████| 74/74 [00:16<00:00,  4.40batch/s, Loss: 0.2560 | Precision: 88.00%]

Epoch 200 | Avg Loss: 0.2702 | Avg Precision: 87.88% | Patience: 1
Training Complete - Best Precision: 88.03%


### 11-13-23

#### Model 3 - Downsample with new features and added more units

In [23]:
model = Gru(8, 500, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.003)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=0, min_lr=.0001, verbose=True)

model3 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model3.state_dict(), '.\\models\\downsample\\gru\\model3.pth')

Train: 100%|████████████████████████| 295/295 [01:06<00:00,  4.45batch/s, Loss: 0.3943 | Precision: 83.97%]


Epoch 1 | Avg Loss: 0.3418 | Avg Precision: 80.96%


Train: 100%|████████████████████████| 295/295 [01:06<00:00,  4.46batch/s, Loss: 0.0894 | Precision: 87.48%]


Epoch 2 | Avg Loss: 0.2641 | Avg Precision: 86.82%


Train: 100%|████████████████████████| 295/295 [01:06<00:00,  4.44batch/s, Loss: 0.0281 | Precision: 89.19%]


...

Epoch 99 | Avg Loss: 0.0187 | Avg Precision: 99.50%


Validate: 100%|███████████████████████| 74/74 [00:13<00:00,  5.50batch/s, Loss: 0.0942 | Precision: 96.55%]


Epoch 99 | Avg Loss: 0.1369 | Avg Precision: 96.53% | Patience: 0


Train: 100%|████████████████████████| 295/295 [01:05<00:00,  4.53batch/s, Loss: 0.0000 | Precision: 99.49%]

Epoch 100 | Avg Loss: 0.0184 | Avg Precision: 99.44%
Training Complete - Best Precision: 96.53%


## Predict | Submit

In [ ]:
ypreds = predict(model1, test_loader)
test_df['Prediction'] = ypreds

In [ ]:
sub = pd.DataFrame()
temp = test_df.copy()
temp["Id"] = temp.apply(lambda x : str(x.Id) + "_" + str(x.Time), axis = 1)
temp['StartHesitation'] = np.where(temp['Prediction'] == 1, 1, 0)
temp['Turn'] = np.where(temp['Prediction'] == 2, 1, 0)
temp['Walking'] = np.where(temp['Prediction'] == 3, 1, 0)
temp = temp[['Id', 'StartHesitation', 'Turn', 'Walking']]
sub = pd.concat([sub, temp])